In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Experimental data

*Roitman, JD and Shadlen, MN (2002), “Response of neurons in the lateral intraparietal area during a combined visual discrimination reaction time task”, Journal of Neuroscience, Vol. 22(21), 9475-9489.*


#### additional information:
- 'coh'      coherence of trial (multiplied by 10 - ie. 32 is a coherence of 3.2%)
- 'correct'  whether the subject was correct (1 - correct, 0 - error)
- rt should be in ms (between 5 and 1762, but most in [200,1200])
- in Shinn et al. animal N (coded as 0 below) is used for all main figures
- per monkey and coherence level: trial number n>500 for animal 0, n>400 for animal 1.



In [ ]:
ddm_data = pd.read_csv("../data/roitman_data_clean.csv")

In [ ]:
ddm_data

# Generative model 
Use the *pyddm* toolbox:

https://pyddm.readthedocs.io/en/stable/

In [ ]:
# load pyddm 

from pyddm import Model
from pyddm.models import DriftConstant, NoiseConstant, BoundConstant, OverlayNonDecision, ICPointSourceCenter
from pyddm.functions import fit_adjust_model, display_model

import pyddm.plot

from pyddm import Fittable
from pyddm.models import LossRobustBIC
from pyddm.functions import fit_adjust_model, fit_model

from pyddm.models import (
    BoundCollapsingExponential,
    DriftLinear,
    ICPointSourceCenter,
    NoiseConstant,
    OverlayNonDecision,
)


from roitman_utils import filter_roitman_data 
# this filters the data and puts it into the format we need for pyddm

In [ ]:
data = filter_roitman_data(ddm_data, 
                            coherence=32, 
                            animal=0 , 
                            n_trial=400, 
                            attach_model_mask=False,
                            partition=None,
                            data_mode='pyddm')

In [ ]:
model_fit = Model(name='Simple model (fitted)',
                        drift= DriftLinear(drift=Fittable(minval=0, maxval=5),t=0, x=Fittable(minval=-20, maxval=10)),
                        noise=NoiseConstant(noise=1),
                        bound=BoundCollapsingExponential(B=Fittable(minval=0.5, maxval=4), tau=Fittable(minval=0.1, maxval=4)),
                        overlay=OverlayNonDecision(nondectime=Fittable(minval=0.1, maxval=0.4)),
                        IC = ICPointSourceCenter(),
                        dx=.001, dt=.01, T_dur=2)

# fit model
fit_adjust_model(data, model_fit,
                fitting_method="differential_evolution",
                lossfunction=LossRobustBIC, verbose=False)

sol = model_fit.solve()


In [ ]:
pyddm.plot.plot_fit_diagnostics(model=model_fit, sample=data)
plt.show()


In [ ]:
# solve model 
sol = model_fit.solve()


In [ ]:
# generate data 
generated_data = sol.resample(k=1000)

In [ ]:
# access trials 
generated_data.choice_lower

generated_data.choice_uper